# 影片轉圖片

### args:
1. `isLog`        : 是否顯示log
2. `case`         : 功能選擇 ( 參考 case_list )
3. `interval`     : 幀數間格 (每幾幀擷取一張影像)
4. `H1_DIR`       : 影片母資料夾路徑
5. `H1_image_DIR` : 影像母資料夾路徑


### 影片資料夾樹狀圖:

說明:
1. for CASE 1 and CASE 2
2. 資料夾可為多階層 ( 以下為範例 )
```
H1_DIR  
├─ H2_FOLDER  
│  ├─ H3_FOLDER  
│  │  ├─ video01.mov  
│  │  ├─ ..  
│  │  └─ videoXX.mov  
│  ├─ H3_FOLDER  
│  ├─ ..  
│  └─ H3_FOLDER  
├─  ..  
└─ H2_FOLDER  
```

---

### CASE 'Video2ImageOneDir' : 輸出1個影像母資料夾

說明:
1. 輸出影像資料夾樹狀圖 ( 階層會與來源相同 ):
```
H1_DIR_video2image
├─ H2_FOLDER
│  ├─ H3_FOLDER
│  │  ├─ video01
│  │  │  ├─ image01.jpg
│  │  │  ├─ ..
│  │  │  ├─ imageXX.jpg
│  │  │  └─ video01.MOV
│  │  ├─ ..
│  │  └─ videoXX
│  ├─ H3_FOLDER
│  ├─ ..
│  └─ H3_FOLDER
├─  ..
└─ H2_FOLDER
```
---

### CASE 'Video2ImageTwoDir' : 輸出2個影像資料夾

說明:
1. 輸出1樹狀圖: 切片影像 ( 輸出的影像序號會接續下去 )
```
H1_DIR_all_image
├─ image01.jpg
├─  ..
└─ imageXX.jpg
```

2. 輸出2樹狀圖: 影像上方有影像出處
```
H1_DIR_info_image
├─ image01.jpg
├─  ..
└─ imageXX.jpg
```
---

### CASE 'CloneImageOneDir' : 克隆影像

說明:
1. 輸入資料夾可為多階層,底層為.jpg檔
2. 輸出影像資料夾: ( 輸出的影像序號會接續下去 )
```
H1_DIR_all_image
├─ image01.jpg
├─  ..  
└─ imageXX.jpg
```
---

### CASE 4 'CloneImageTwoDir' : 克隆影像ver2

說明:
1. 輸入資料夾可為多階層,底層為.jpg檔
2. 輸出影像資料夾: ( 輸出的影像序號會接續下去 )
```
H1_DIR_all_image
├─ image01.jpg
├─  ..  
└─ imageXX.jpg
```
3. 輸出影像 ( 影像上方有影像出處資訊 ) 資料夾:
```
H1_DIR_info_image
├─ image01.jpg
├─  ..  
└─ imageXX.jpg
```

In [ ]:
import cv2
import os
from tqdm.notebook import tqdm # jupyter notebook的進度條

case_list = {'Video2ImageOneDir' : 0,
             'Video2ImageTwoDir' : 1,
             'CloneImageOneDir'  : 2,
             'CloneImageTwoDir'  : 3}


isLog = 1
case = case_list['CloneImageOneDir']
interval = 10
H1_DIR = 'C:\\Users\\danielwu\\Desktop\\220316'         # Just Def When Case 0 or Case 1
H1_image_DIR = 'C:\\Users\\danielwu\\Desktop\\newtmp'   # Just Def When Case 2 or Case 3

## @brief Description: 取得CMD大小
#  @param None
#  
#  @return col
#  @date 20220318  danielwu
def GetCmdSize():
    import shutil
    col, _ = shutil.get_terminal_size()
    col = col - 21
    return col



## @brief Description: 建立資料夾
#  @param folder
#  
#  @return None
#  @date 20220321  danielwu
def MakeDirs(folder):
    if not os.path.isdir(folder):
        os.makedirs(folder)



## @brief Description: 取得母資料夾內所有特定格式檔案路徑
#  @param [in] folder     母資料夾路徑
#  @param [in] extension  副檔名
#  
#  @return paths
#  @date 20220321  danielwu
def GetPaths(folder, extension='.MOV'):
    paths = []

    for response in os.walk(folder):        # response = (dirpath, dirname, filenames)
        if response[2] != []:               # look for filenames
            for f in response[2]:
                if f.endswith(extension):  # only append .MOV file in paths 
                    paths.append(os.path.join(response[0], f))
    
    return paths



## @brief Description: 影片轉成影像
#
#  @param [in] video_path  影片路徑
#  @param [in] save_dir    儲存影像之路徑
#  @param [in] interval    幀數間格
#
#  @return None
#  @date  20220318  danielwu 
def CutVideo(video_path, save_dir, interval=1):
    frame_count = 0  # 保存幀的索引
    frame_index = 0  # 原影片的幀樹索引 ( interval * frame_count = frame_index )
    
    cap = cv2.VideoCapture(video_path)

    if cap.isOpened():
        ret = True
    else:
        ret = False
        print('{} 讀取失敗!\n'.format(video_path))

    while(ret):
        ret, frame = cap.read()
        if ret == True and frame_index % interval == 0:
            cv2.imwrite(os.path.join(save_dir, '{}.jpg'.format(frame_count).zfill(10)), frame)
            frame_count += 1
        frame_index += 1
        
    cap.release()



## @brief Description: 影片轉成影像
#                      (一個資料夾為全部存在一起,另一個同為將全部存在一起且將路徑寫在影像上方)
#
#  @param [in] video_path  影片路徑
#  @param [in] frame_count 儲存影像之編號
#  @param [in] save_dir1   儲存影像之路徑
#  @param [in] save_dir2   儲存影像之路徑
#  @param [in] interval    幀數間格
#
#  @return frame_count
#  @date  20220318  danielwu 
def CutVideoVer2(video_path, frame_count, save_dir1, save_dir2, interval=1):
    frame_index = 0
    
    cap = cv2.VideoCapture(video_path)

    if cap.isOpened():
        ret = True
    else:
        ret = False
        print('{} 讀取失敗!\n'.format(video_path))

    while(ret):
        ret, frame = cap.read()
        if ret == True and frame_index % interval == 0:
            # 儲存影片 -> 影像
            cv2.imwrite(os.path.join(save_dir1, '{}.jpg'.format(frame_count).zfill(10)), frame)
            
            # 儲存影片 -> (影片路徑資訊) -> 影像
            text = '{}{}'.format(H1_DIR.split('\\')[-1], video_path.split(H1_DIR)[1])  # 影像出處
            frame = cv2.putText(frame, text, (10, 35), cv2.FONT_HERSHEY_DUPLEX, 0.8, (40, 135, 255), 1, cv2.LINE_AA)
            cv2.imwrite(os.path.join(save_dir2, '{}.jpg'.format(frame_count).zfill(10)), frame)
            frame_count += 1
        frame_index += 1
        
    cap.release()
    return frame_count



## @brief Description: 儲存影像
#  @param [in] folder  儲存影像資料夾路徑
#  @param [in] image   要儲存的影像
#  
#  @return None
#  @date 20220321  danielwu
def SaveImage(folder, count, image):
    cv2.imwrite(os.path.join(folder, '{}.jpg'.format(count).zfill(10)), image)



if __name__ == '__main__':
    if isLog:
        col = GetCmdSize()
    
    # Case 0 & 1
    if 0 <= case < 2:
        # Get Video Paths from H1_DIR (Video is .MOV file)
        video_paths = GetPaths(H1_DIR, extension='.MOV')
        if isLog:
            print('== Info ' + '='*(col-8))
            print('There are {} videos in dir: {}'.format(len(video_paths), H1_DIR))
            print('='*col)
    
    # Case 0
    if case == 0:
        import shutil
        for video_path in tqdm(video_paths):
            tmp1 = '{}_video2image'.format(H1_DIR)
            tmp2 = video_path.split(H1_DIR+'\\')[1].split('.')[0]
            save_path = os.path.join(tmp1, tmp2)
            MakeDirs(save_path)

            CutVideo(video_path, save_path, interval)
            shutil.copy(video_path, save_path)
    
    # Case 1
    if case == 1:
        frame_count = 0
        save_dir1 = '{}_all_image'.format(H1_DIR)
        save_dir2 = '{}_info_image'.format(H1_DIR)
        for f in [save_dir1, save_dir2]:
            MakeDirs(f)
        for video_path in tqdm(video_paths):
            frame_count = CutVideoVer2(video_path, frame_count, save_dir1, save_dir2, interval)
    
    # Case 2 & 3
    if 2 <= case < 4:
        # Get Image Paths from H1_image_DIR (Image is .jpg file)
        image_paths = GetPaths(H1_image_DIR, extension='.jpg')
        if isLog:
            print('== Info ' + '='*(col-8))
            print('There are {} images in dir: {}'.format(len(image_paths), H1_image_DIR))
            print('='*col)

        # Case 2 & 3
        image_count = 0
        save_dir1 = '{}_all_image'.format(H1_image_DIR)
        MakeDirs(save_dir1)
        ## Case 3
        if case == 3:
            save_dir2 = '{}_info_image'.format(H1_image_DIR)
            MakeDirs(save_dir2)
        # Case 2 & 3
        for image_path in tqdm(image_paths):
            image = cv2.imread(image_path)
            SaveImage(save_dir1, image_count, image)
            ## Case 3
            if case == 3:    
                text = '{}{}'.format(H1_image_DIR.split('\\')[-1], image_path.split(H1_image_DIR)[1])  # 影像出處
                image = cv2.putText(image, text, (10, 35), cv2.FONT_HERSHEY_DUPLEX, 0.8, (40, 135, 255), 1, cv2.LINE_AA)
                SaveImage(save_dir2, image_count, image)
            image_count = image_count + 1


